In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein
from Bio.Align.Applications import MuscleCommandline
from pandas import DataFrame
from ggplot import *

import re

%load_ext rpy2.ipython


CAP45_record = SeqIO.read('../sequence_files/CAP45.gb', 'genbank')
Du156_record = SeqIO.read('../sequence_files/Du156.gb', 'genbank')
HXB2_record = SeqIO.read('../sequence_files/HXB2.gb', 'genbank')

In [2]:
# Select out env
CAP45_env_aa = CAP45_record.features[4].qualifiers['translation'][0]
Du156_env_aa = Du156_record.features[6].qualifiers['translation'][0]
HXB2_env_aa = HXB2_record.features[15].qualifiers['translation'][0]

In [3]:
def Build_Ref_dict(origin, ref):
    #write seqs to file
    origin_rec = SeqRecord(Seq(origin, generic_protein),
                           id = 'origin', 
                           description = '')
    ref_rec = SeqRecord(Seq(ref, generic_protein),
                        id = 'ref',
                        description = '')
    SeqIO.write([origin_rec, ref_rec], 
            'input.fasta', 'fasta')
    
    # Align with muscle
    mc = MuscleCommandline(input='input.fasta', out='output.fasta')
    mc()
    
    # read alignment
    ali = [seq_record.seq for seq_record in SeqIO.parse('output.fasta', 
                                                        'fasta')]
    origin = str(ali[0])
    ref = str(ali[1])
    
    # Build dict
    l = len(origin)
    origin_pos = 0
    num_pos = 0
    gap_pos = 0
    
    ref_dict = dict()
    for i in range(0,l):
        if ref[i] != '-':
            num_pos += 1
            gap_pos = 0
        if ref[i] == '-':
            gap_pos += 1
        
        if origin[i] != '-' and gap_pos != 0:
            origin_pos += 1
            ref_dict[origin_pos] = str(num_pos) + '_' + str(gap_pos)
        if origin[i] != '-' and gap_pos == 0:
            origin_pos += 1
            ref_dict[origin_pos] = str(num_pos)
    return(ref_dict)


In [4]:
# Find CAP45_HXB2 dict
CAP45_HXB2 = Build_Ref_dict(CAP45_env_aa, HXB2_env_aa)

# PNGS in CAP45
PNGS = []
matches = re.finditer('N(?=[A-OQ-Z][TS])', CAP45_env_aa)
for match in matches:
    PNGS.append({'name':'CAP45' , 'pos':CAP45_HXB2[match.start()+1], 'pres':0})
    print match.start()+1,CAP45_HXB2[match.start()+1]

87 88
132 133
136 144
145 156
149 160
181 190_2
191 197
224 230
228 234
235 241
256 262
270 276
283 289
295 301
327 334
328 335
332 339
348 356
378 386
388 403
391 406
422 442
426 446
444 463
592 611
597 616
606 625
618 637
655 674


In [9]:
# Find Du156_HXB2 dict
Du156_HXB2 = Build_Ref_dict(Du156_env_aa, HXB2_env_aa)

# PNGS in Du156
matches = re.finditer('N(?=[A-OQ-Z][TS])', Du156_env_aa)
for match in matches:
    PNGS.append({'name':'Du156' , 'pos':Du156_HXB2[match.start()+1], 'pres':0})
    #print match.start()+1, Du156_HXB2[match.start()+1]

In [6]:
# PNGS in HXB2
matches = re.finditer('[N](?=[A-OQ-Z][TS])', HXB2_env_aa)
for match in matches:
    PNGS.append({'name':'HXB2' , 'pos':match.start()+1, 'pres':1})
    #print(match.start()+1)

In [7]:
PNGS_data = DataFrame(PNGS)

In [72]:
%%R -i PNGS_data

library(ggplot2)
library(grid)

levels <- c('88', levels(factor(PNGS_data$pos))[1:length(PNGS_data$pos)-1])
PNGS_data$pos_f <- factor(PNGS_data$pos, ordered = T, levels = levels)
ranges <- data.frame(xmin = c(0,1.5,9.5,13.5,21.5,22.5,28.5,34.5,37.5, 40.5),
                     xmax = c(1.5,9.5,13.5,21.5,22.5,28.5,34.5,37.5,40.5, 48.5),
                     ymin = rep(1,10),
                     ymax = rep(4.1,10))
labels <- data.frame(x = c(1,5.5,11.5,17.5,22,31.5,25.5,36,39,44.5,50.5, 50.5, 50.5), 
                     y = c(rep(4.3,10),3.5, 2.5, 1.5), 
                     label = c('C1','V1','V2','C2','V3','V4','C3','C4','V5', 'GP41', '31', '29', '29'))

ggplot(data = PNGS_data) + 
 geom_rect(aes(xmin = as.numeric(factor(pos_f)) - 0.4, 
               xmax = as.numeric(factor(pos_f)) + 0.4, 
               ymin = as.numeric(factor(name)), 
               ymax = as.numeric(factor(name)) + 1,
               fill = name)) +
 geom_rect(aes(xmin = xmin, 
               xmax = xmax, 
               ymin = ymin, 
               ymax = ymax),
               fill = NA, col = 'black', data = ranges) +
 geom_text(aes(x = x, y = y, label = label), data = labels) +
 scale_x_continuous('HXB2 position',
                    breaks = 1:48,
                    labels = levels(factor(PNGS_data$pos_f)),
                    limits = c(-0.5,52),
                    expand = c(0,0)) +
 scale_y_continuous('',
                    limits = c(1,4.5),
                    breaks=NULL,
                    expand = c(0,0)) +
 ggtitle('PNGS locations') +
 theme_bw() +
 theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
       plot.margin = unit(c(0,0,0,0),'mm'),
       legend.title = element_blank())
 
ggsave(file = '../Graphs/PNGS_ggplot.png', width = 8, height = 3)

In [22]:
input_file = open('/Users/roux-cilferreira/Desktop/naccess/CAP45_tri.rsa')
CAP45_sa = dict()
i = 0
for line in input_file:
    if not line.startswith('REM'):
        line_list = line.split()
        CAP45_sa[line_list[3]] = float(line_list[4])